SETUP DATA INGESTION

In [0]:
%python
dbutils.entry_point.getDbutils().notebook().getContext().notebookPath().get()

In [0]:
%python
# Recreate Schema for prime run of the job
dbutils.notebook.run("/Workspace/KungfuPandas/Setup/Schema_Setup", timeout_seconds=300)

In [0]:
CREATE OR REPLACE TEMPORARY VIEW patient_details_csv
USING CSV
OPTIONS (
  path 's3://kungfupandas/landing_zone/prime_patient_details_10k.csv',
  header 'true',
  inferSchema 'false'
);

CREATE OR REPLACE TABLE medical_research_recalls.bronze.kgf_patient_details
USING DELTA
LOCATION 's3://kungfupandas/bronze/kgf_patient_details'
AS 
SELECT
  `id`,
  `First Name` AS first_name,
  `Surname` AS surname,
  `Title` AS title,
  `Date of Birth` AS date_of_birth,
  `Address` AS address,
  `Mobile` AS mobile,
  `Home Phone` AS home_phone,
  `Work Phone` AS work_phone
FROM patient_details_csv;

select * from medical_research_recalls.bronze.kgf_patient_details limit 10;

In [0]:
CREATE OR REPLACE TEMPORARY VIEW patient_demographics_csv
USING CSV
OPTIONS (
  path 's3://kungfupandas/landing_zone/prime_patient_demographics_10k.csv',
  header 'true',
  inferSchema 'false'
);

CREATE OR REPLACE TABLE medical_research_recalls.bronze.kgf_patient_demographics
USING DELTA
LOCATION 's3://kungfupandas/bronze/kgf_patient_demographics'
AS 
SELECT
  `id`,
  `Gender` AS gender,
  `Marital Status` AS marital_status,
  `Employment Status` AS employment_status,
  `Education Level` AS education_level,
  `Ethnicity` AS ethnicity,
  `Annual Income (AUD)` AS annual_income_aud,
  `Insurance Type` AS insurance_type,
  `Primary Language` AS primary_language 
FROM patient_demographics_csv;

select * from medical_research_recalls.bronze.kgf_patient_demographics limit 10;

In [0]:
CREATE OR REPLACE TEMPORARY VIEW patient_cancer_risk_csv
USING CSV
OPTIONS (
  path 's3://kungfupandas/landing_zone/prime_patient_cancer_risk_10k.csv',
  header 'true',
  inferSchema 'false'
);

CREATE OR REPLACE TABLE medical_research_recalls.bronze.kgf_patient_cancer_risk
USING DELTA
LOCATION 's3://kungfupandas/bronze/kgf_patient_cancer_risk'
AS 
SELECT
  `id`,
  `smoking`,
  `alcohol_consumption`,
  `family_history`,
  `poor_diet`,
  `obesity`,
  `sedentary_lifestyle`,
  `exposure_to_carcinogens`,
  `chronic_inflammation`,
  `HPV_infection` AS hpv_infection,
  `sun_exposure`,
  `air_pollution`,
  `radiation_exposure`,
  `chemical_exposure`,
  `immunosuppression`,
  `night_shift_work`,
  `Age` AS age,
  `Gender` AS gender
FROM patient_cancer_risk_csv;

select * from medical_research_recalls.bronze.kgf_patient_cancer_risk limit 10;

In [0]:
CREATE OR REPLACE TEMPORARY VIEW patient_cancer_detection_csv
USING CSV
OPTIONS (
  path 's3://kungfupandas/landing_zone/prime_patient_cancer_detection_10k.csv',
  header 'true',
  inferSchema 'false'
);

CREATE OR REPLACE TABLE medical_research_recalls.bronze.kgf_patient_cancer_detection
USING DELTA
LOCATION 's3://kungfupandas/bronze/kgf_patient_cancer_detection'
AS 
SELECT
  `Patient ID` AS patient_id,
  `Detection Date` AS detection_date,
  `Detection Method` AS detection_method,
  `Stage at Detection` AS stage_at_detection,
  `Cancer Type` AS cancer_type,
  `Cancer Probability` AS cancer_probability
FROM patient_cancer_detection_csv;

select * from medical_research_recalls.bronze.kgf_patient_cancer_detection limit 10;

In [0]:
CREATE OR REPLACE TEMPORARY VIEW patient_risk_scores_csv
USING CSV
OPTIONS (
  path 's3://kungfupandas/landing_zone/prime_patient_risk_scores_10k.csv',
  header 'true',
  inferSchema 'false'
);

CREATE OR REPLACE TABLE medical_research_recalls.bronze.kgf_patient_risk_scores
USING DELTA
LOCATION 's3://kungfupandas/bronze/kgf_patient_risk_scores'
AS 
SELECT
  `id`,
  `age`,
  `gender`,
  `smoking_score`,
  `alcohol_consumption_score`,
  `family_history_score`,
  `poor_diet_score`,
  `obesity_score`,
  `sedentary_lifestyle_score`,
  `exposure_to_carcinogens_score`,
  `chronic_inflammation_score`,
  `hpv_infection_score`,
  `sun_exposure_score`,
  `air_pollution_score`,
  `radiation_exposure_score`,
  `chemical_exposure_score`,
  `immunosuppression_score`,
  `night_shift_work_score`,
  `total_risk_score`,

  -- Scale all scores by 10 to allow for more accurate processing
  `lung_cancer` * 10 AS lung_cancer,
  `breast_cancer` * 10 AS breast_cancer,
  `colon_cancer` * 10 AS colon_cancer,
  `skin_cancer` * 10 AS skin_cancer,
  `cervical_cancer` * 10 AS cervical_cancer,
  `prostate_cancer` * 10 AS prostate_cancer,
  `leukemia` * 10 AS leukemia,
  `pancreatic_cancer` * 10 AS pancreatic_cancer

FROM patient_risk_scores_csv;

select * from medical_research_recalls.bronze.kgf_patient_risk_scores limit 10;

In [0]:
-- Add comments for Simulting Lineage

COMMENT ON TABLE medical_research_recalls.bronze.kgf_patient_details IS 'Patient Details bronze tier has been Ingested via /notebooks from Amazon S3 landing zone. Dataset from csv: patient_details_10k.csv';
COMMENT ON TABLE medical_research_recalls.bronze.kgf_patient_demographics IS 'Patient Demographics bronze tier has been Ingested via /notebooks from Amazon S3 landing zone. Dataset from csv: patient_demographics_10k.csv';
COMMENT ON TABLE medical_research_recalls.bronze.kgf_patient_cancer_risk IS 'Patient Cancer Risk bronze tier has been Ingested via /notebooks from Amazon S3 landing zone. Dataset from csv: patient_cancer_detection_10k.csv';
COMMENT ON TABLE medical_research_recalls.bronze.kgf_patient_cancer_detection IS 'Patient Cancer Detection bronze tier has been Ingested via /notebooks from Amazon S3 landing zone. DataSet from csv: patient_cancer_detection_10k.csv';
COMMENT ON TABLE medical_research_recalls.bronze.kgf_patient_cancer_detection IS 'Patient Cancer Scores bronze tier has been Ingested via /notebooks from Amazon S3 landing zone. DataSet from csv: patient_risk_scores_10k.csv';